In [3]:
# библиотеки
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import UpSampling2D, Reshape, Flatten, Conv2DTranspose
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np

2024-06-23 02:48:51.086390: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 02:48:51.088549: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 02:48:51.118969: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 02:48:51.685718: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# загрузка датасета
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
# предподготовка данных
# нормируем и приведем к требуемому формату данные
x_train_conv = x_train.astype('float32') / 255.
x_test_conv  = x_test.astype('float32') / 255.
x_train_conv = np.expand_dims(x_train_conv, -1)
x_test_conv  = np.expand_dims(x_test_conv, -1)
print('x_train_conv.shape =', x_train_conv.shape)
print('x_test_conv.shape  =', x_test_conv.shape)

x_train_conv.shape = (60000, 28, 28, 1)
x_test_conv.shape  = (10000, 28, 28, 1)


In [6]:
# нейронная сеть
img_input = Input((28,28,1))

x = Conv2D(32, (3, 3), padding='same', activation='relu')(img_input)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
z = MaxPooling2D()(x)

x = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', activation='relu')(z)
x = BatchNormalization()(x)

x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)

x = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)

x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

model = Model(img_input, x)

In [7]:
# компиляция
model.compile(
    loss='mse',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

In [8]:
# callbacks
es = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
mch = ModelCheckpoint(
    "autoencoder.keras",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch'
)

In [9]:
# обучение
history = model.fit(
    x = x_train_conv,
    y = x_train_conv,
    batch_size = 128,
    epochs = 100,
    validation_data = (x_test_conv, x_test_conv),
    verbose = 1,
    callbacks = [mch, es]
)

Epoch 1/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.7317 - loss: 0.0944
Epoch 1: val_loss improved from inf to 0.01413, saving model to autoencoder.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 154s 322ms/step - accuracy: 0.7318 - loss: 0.0943 - val_accuracy: 0.8132 - val_loss: 0.0141
Epoch 2/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.8149 - loss: 0.0097
Epoch 2: val_loss improved from 0.01413 to 0.00588, saving model to autoencoder.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 145s 309ms/step - accuracy: 0.8149 - loss: 0.0097 - val_accuracy: 0.8143 - val_loss: 0.0059
Epoch 3/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 0.8155 - loss: 0.0056
Epoch 3: val_loss improved from 0.00588 to 0.00429, saving model to autoencoder.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 147s 314ms/step - accuracy: 0.8155 - loss: 0.0056 - val_accuracy: 0.8144 - val_loss: 0.0043
Epoch 4/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.8151 - loss: 0.0039
Epoch 4: val_loss improved 

In [10]:
# копируем полученную модель на s3-хранилище
!aws --endpoint-url=https://storage.yandexcloud.net/ s3 cp autoencoder.keras s3://cherepanov-bucket/autoencoder.keras

upload: ./autoencoder.keras to s3://cherepanov-bucket/autoencoder.keras
